In [80]:
COST_INDEX = 0.07
SELL_TAX = 0.036
LOCATION = 60003760
REGION = 10000002
DURATION_MODIFIER = (1 - 0.32) * (1 - 0.15)
BP_LIST = """Capital Quantum Microprocessor Blueprint	10	0	-1	Advanced Capital Construction Components
Capital Crystalline Carbonide Armor Plate Blueprint	0	0	-1	Advanced Capital Construction Components
Capital Compact Pb-Acid Cap Battery Blueprint	0	0	-1	Capacitor Battery
Cap Booster 150 Blueprint	10	20	-1	Capacitor Booster Charge
Skirmish Command Burst I Blueprint	0	0	-1	Command Burst
Shield Command Burst I Blueprint	0	0	-1	Command Burst
Sustained Shield Emitter Blueprint	10	20	-1	Construction Components
Caldari Control Tower Small Blueprint	10	16	5	Control Tower
Caldari Control Tower Small Blueprint	10	16	5	Control Tower
Minmatar Control Tower Small Blueprint	0	0	-1	Control Tower
Amarr Control Tower Small Blueprint	0	0	-1	Control Tower
Gallente Control Tower Small Blueprint	0	0	-1	Control Tower
Corporate Hangar Array Blueprint	7	0	-1	Corporate Hangar Array
Capital Gremlin Compact Energy Neutralizer Blueprint	0	0	-1	Energy Neutralizer
Capital Infectious Scoped Energy Neutralizer Blueprint	0	0	-1	Energy Neutralizer
Warfare Computation Core Blueprint	10	20	-1	Hybrid Tech Components
Optimized Nano-Engines Blueprint	10	20	-1	Hybrid Tech Components
Metallofullerene Plating Blueprint	0	0	-1	Hybrid Tech Components
Superconducting Ladar Amplifier Blueprint	10	20	-1	Hybrid Tech Components
Reconfigured Subspace Calibrator Blueprint	10	0	-1	Hybrid Tech Components
Light Hull Maintenance Bot I Blueprint	10	20	-1	Logistic Drone
EMP M Blueprint	0	0	-1	Projectile Ammo
50000MN Cold-Gas Enduring Microwarpdrive Blueprint	0	0	-1	Propulsion Module
10000MN Monopropellant Enduring Afterburner Blueprint	0	0	-1	Propulsion Module
Capital Coaxial Compact Remote Armor Repairer Blueprint	0	0	-1	Remote Armor Repairer
Capital I-ax Enduring Remote Armor Repairer Blueprint	0	0	-1	Remote Armor Repairer
Large Thermal Armor Reinforcer I Blueprint	8	0	-1	Rig Armor
Small Processor Overclocking Unit I Blueprint	10	0	-1	Rig Core
Small Command Processor I Blueprint	0	0	-1	Rig Core
Small Energy Locus Coordinator I Blueprint	0	0	-1	Rig Energy Weapon
Small Hyperspatial Velocity Optimizer I Blueprint	10	0	-1	Rig Navigation
Small Low Friction Nozzle Joints I Blueprint	10	0	-1	Rig Navigation
Medium Low Friction Nozzle Joints I Blueprint	0	0	-1	Rig Navigation
Small Signal Focusing Kit I Blueprint	10	0	-1	Rig Scanning
Capital Core Defense Operational Solidifier I Blueprint	0	0	-1	Rig Shield
Large Core Defense Operational Solidifier I Blueprint	0	0	-1	Rig Shield
Scan Rangefinding Array I Blueprint	10	0	-1	Scanning Upgrade
Large Shield Extender I Blueprint	0	0	-1	Shield Extender
Ship Maintenance Array Blueprint	7	0	-1	Ship Maintenance Array
Caldari Shuttle Blueprint	7	10	-1	Shuttle
Ice Harvester I Blueprint	8	0	-1	Strip Miner
Standup Ladar ECM Script Blueprint	10	20	-1	Structure ECM script
Optimal Range Disruption Script Blueprint	8	0	-1	Tracking Disruption Script
Nova XL Torpedo Blueprint	8	16	-1	XL Torpedo"""

In [81]:
from src.util import connect_to_db

conn = await connect_to_db()

bp_info = await conn.fetch('SELECT * FROM sde.blueprints')
bp_materials = await conn.fetch("SELECT * FROM sde.blueprint_materials WHERE activity = 'manufacturing'")
bp_products = await conn.fetch("SELECT * FROM sde.blueprint_products WHERE activity = 'manufacturing'")
bp_products = {bp['blueprint_type_id']: {'type_id':bp['product_type_id'], 'quantity': bp['quantity']} for bp in bp_products}

type_ids = dict(await conn.fetch('SELECT type_id, name FROM sde.type_ids'))
type_names = dict(await conn.fetch('SELECT name, type_id FROM sde.type_ids'))
type_published = dict(await conn.fetch('SELECT type_id, published FROM sde.type_ids'))

market_buy = dict(await conn.fetch(f'SELECT type_id, buy_max FROM market.aggregates WHERE location_id = {LOCATION}'))
market_sell = dict(await conn.fetch(f'SELECT type_id, sell_min FROM market.aggregates WHERE location_id = {LOCATION}'))
market_region_sell = dict(await conn.fetch(f'SELECT type_id, sell_min FROM market.aggregates WHERE region_id = {REGION}'))

blueprints = {}
for bp in bp_info:
    if type_published[bp['blueprint_type_id']] == False:
        continue
    if bp['blueprint_type_id'] not in bp_products:
        continue
    blueprints[bp['blueprint_type_id']] = {
        'name': type_ids[bp['blueprint_type_id']],
        'manufacture_time': bp['manufacturing_time'],
        'materials': [],
        'product': bp_products[bp['blueprint_type_id']],
    }
for bp_mat in bp_materials:
    if bp_mat['blueprint_type_id'] not in blueprints:
        continue
    blueprints[bp_mat['blueprint_type_id']]['materials'].append({
        'type_id': bp_mat['material_type_id'],
        'quantity': bp_mat['quantity'],
    })

In [82]:
import math
from decimal import Decimal

SELL_TAX = Decimal(SELL_TAX)
COST_INDEX = Decimal(COST_INDEX)

sell_price = {}
buy_price = {}
profit = {}
profit_per_day = {}
profitable_price = {}
bp_price = {}

owned_mat_eff = {}
owned_time_eff = {}
for bp_line in BP_LIST.split("\n"):
    bp_name, bp_mat_eff, bp_time_eff, _, _ = bp_line.split("\t")
    owned_mat_eff[type_names[bp_name]] = int(bp_mat_eff) / 100
    owned_time_eff[type_names[bp_name]] = int(bp_time_eff) / 100

for bp_id, bp in blueprints.items():
    if bp['product']['type_id'] not in market_buy or market_buy[bp['product']['type_id']] is None:
        continue
    if bp['product']['type_id'] not in market_sell or market_sell[bp['product']['type_id']] is None:
        continue
    
    bp_price[bp_id] = market_region_sell[bp_id] if bp_id in market_region_sell else 0
    bp_price[bp_id] = bp_price[bp_id] if bp_price[bp_id] is not None else 0
    
    buy_price[bp_id] = 0
    cant_buy = False
    for material in bp['materials']:
        if material['type_id'] not in market_sell or market_sell[material['type_id']] is None:
            cant_buy = True
            break
        buy_quantity = math.ceil(material['quantity'] * (1 - owned_mat_eff[bp_id] if bp_id in owned_mat_eff else 1))
        buy_price[bp_id] += market_sell[material['type_id']] * buy_quantity
        
    if cant_buy:
        continue
    
    profitable_price[bp_id] = buy_price[bp_id] / (1 - COST_INDEX) / (1 - SELL_TAX) / bp['product']['quantity']
    
    sell_price[bp_id] = market_buy[bp['product']['type_id']] * bp['product']['quantity']
    # sell_price[bp_id] = market_sell[bp['product']['type_id']] * bp['product']['quantity']
    profit[bp_id] = sell_price[bp_id] - buy_price[bp_id] - sell_price[bp_id] * SELL_TAX - buy_price[bp_id] * COST_INDEX
    profit_per_day[bp_id] = profit[bp_id] * Decimal(86400 / (bp['manufacture_time'] * DURATION_MODIFIER * (1 - owned_time_eff[bp_id] if bp_id in owned_time_eff else 1)))

print(f"{'Name':<64} {'Profit per day':>16} {'Profit':>16} {'Buy price':>16} {'Sell price':>16} {'Runs/BP':>8} {'Days/BP':>8}")
for bp_id, bp_profit in sorted(profit_per_day.items(), key=lambda item: item[1], reverse=True):
    if bp_price[bp_id] == 0:
        continue
    if bp_id in owned_mat_eff:
        continue
    print(f"{type_ids[bp_id]:64} {bp_profit:16,.2f} {profit[bp_id]:16,.2f} {buy_price[bp_id]:16,.2f} {sell_price[bp_id]:16,.2f} {bp_price[bp_id] / profit[bp_id]:8.2f} {bp_price[bp_id] / profit_per_day[bp_id]:8.2f}")
    

Name                                                               Profit per day           Profit        Buy price       Sell price  Runs/BP  Days/BP
Magnetometric-FTL Interlink Communicator Blueprint                 301,855,747.55    32,309,744.83   124,674,631.00   171,900,000.00    61.78     6.61
Radar-FTL Interlink Communicator Blueprint                         113,987,916.40    12,200,928.83   129,503,431.00   156,400,000.00   176.22    18.86
'Azmaru' Electromagnetic Disruptive Lance Blueprint                 48,746,782.01    23,479,700.00   232,210,000.00   282,100,000.00    22.13    10.66
Personal Hangar Array Blueprint                                     14,479,272.60     1,162,363.83    16,941,379.60    20,010,000.00   258.09    20.72
Medium Industrial Core II Blueprint                                  9,921,405.38     2,329,670.00    43,383,000.00    50,570,000.00   111.60    26.21
Small Beam Laser Battery Blueprint                                   6,383,669.48       256,23

In [83]:
print(f"{'Name':<64} {'Profit / day':>16} {'Profit / run':>16} {'Buy price':>16} {'Sell price':>16} {'Profitable price':>16}")
for bp_id, bp_profit in sorted(profit_per_day.items(), key=lambda item: item[1], reverse=True):
    if bp_id in owned_mat_eff:
        print(f"{type_ids[bp_id]:64} {bp_profit:16,.2f} {profit[bp_id]:16,.2f} {buy_price[bp_id]:16,.2f} {sell_price[bp_id]:16,.2f} {profitable_price[bp_id]:16,.2f}")

Name                                                                 Profit / day     Profit / run        Buy price       Sell price Profitable price
Capital Core Defense Operational Solidifier I Blueprint             14,113,764.87       566,510.84    21,678,588.00    24,650,000.00    24,180,819.17
10000MN Monopropellant Enduring Afterburner Blueprint               12,990,137.02     3,128,458.00    61,078,600.00    71,040,000.00    68,128,541.47
Shield Command Burst I Blueprint                                     6,684,258.15       160,979.22       557,866.90       786,200.00       622,258.18
Small Energy Locus Coordinator I Blueprint                           5,376,399.84        53,950.68        66,700.30       130,000.00        74,399.12
Medium Low Friction Nozzle Joints I Blueprint                        2,793,688.39        56,067.77       672,131.80       804,200.00       749,712.00
EMP M Blueprint                                                      2,138,670.21         8,584.38  